In [1]:
import pandas as pd
from more_itertools import chunked
from requests import get, post
from tqdm import tqdm
import yaml

N_DEVIATIONS = 10_000

with open('config.yml') as config_file:
    config = yaml.safe_load(config_file)

def get_access_token():
    response = post('https://www.deviantart.com/oauth2/token', {
        'grant_type': 'client_credentials',
        'client_id': config['client_id'],
        'client_secret': config['client_secret'],
    })
    data = response.json()
    access_token = data['access_token']

    return access_token

access_token = get_access_token()

In [2]:
def browse_latest(offset=0):
    response = get('https://www.deviantart.com/api/v1/oauth2/browse/newest', {
        'access_token': access_token,
        'q': 'furry',
        'limit': 120,
        'offset': offset,
        'mature_content': 'true'
    })
    data = response.json()

    return data

offset = 0
deviations = {}
while len(deviations) < N_DEVIATIONS:
    response = browse_latest(offset)
    for deviation in response['results']:
        deviations[deviation['deviationid']] = deviation

    if not response['has_more']:
        break

    offset = response['next_offset']

print(f'Gathered {len(deviations)} deviations')

Gathered 5008 deviations


In [3]:
def get_metadata(ids):
    response = get('https://www.deviantart.com/api/v1/oauth2/deviation/metadata', {
        'access_token': access_token,
        'deviationids[]': ids,
        'mature_content': 'true'
    })
    data = response.json()['metadata']

    return data

for chunk in tqdm(list(chunked(deviations.values(), 25))):
    ids = list(map(lambda x: x['deviationid'], chunk))
    metadatas = get_metadata(ids)
    for metadata in metadatas:
        deviations[metadata['deviationid']]['tags'] = list(map(lambda x: x['tag_name'], metadata['tags']))

100%|██████████| 201/201 [01:07<00:00,  2.96it/s]


In [4]:
tags = []
for deviation in deviations.values():
    tags.extend(deviation['tags'])

tags = pd.Series(tags)
for name, count in tags.value_counts(ascending=False).iloc[:100].items():
    print(f'{name}: {count}')

furry: 4600
anthro: 1744
furryanthro: 1085
furryart: 1044
oc: 655
cute: 578
art: 576
digitalart: 561
furryartist: 535
anthropomorphic: 500
female: 458
commission: 452
furrycharacter: 448
halloween: 446
furryfemale: 432
wolf: 406
cat: 401
fursona: 394
furryfandom: 377
originalcharacter: 371
adopt: 364
digital: 329
adoptable: 322
fox: 312
dog: 299
anthrofurry: 270
furrycommission: 266
canine: 263
commissionsopen: 249
male: 245
ych: 219
dragon: 211
sexy: 209
furryfurries: 209
nsfw: 200
girl: 196
character: 192
anime: 192
fanart: 189
sketch: 185
drawing: 185
digitaldrawing: 185
feral: 177
boobs: 175
feline: 174
anthrocharacter: 170
pokemon: 165
adoptablesopen: 163
adoptables: 162
furries: 160
comic: 159
cartoon: 157
animal: 156
characterdesign: 150
illustration: 150
fantasy: 149
fluffy: 149
chibi: 146
artwork: 137
furryoc: 135
furryartwork: 126
anthroart: 122
auction: 122
commissions: 117
furryadoptable: 117
digitalpainting: 115
sfw: 111
nude: 111
open: 109
demon: 108
commissionart: 105
wo